In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.insert(0,"../")

In [ ]:
from torch.utils.data import DataLoader
import torch

In [ ]:
import random
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
from core.model.model import MoCo_scene_and_view as MoCo
from core.dataloader import CLEVR_train, collate_boxes, CLEVR_train_onlyquery, collate_boxes_onlyquery
from core.utils import compute_features
from core.dataloader import sample_same_scene_negs
from vis_utils import *

In [ ]:
# kmeans_train_dataset = CLEVR_train_onlyquery(root_dir='/home/mprabhud/dataset/clevr_lang/npys/aa_5v.txt')
# kmeans_train_loader = DataLoader(kmeans_train_dataset, batch_size=16, shuffle=False, collate_fn=collate_boxes_onlyquery)

query_train_dataset = CLEVR_train(root_dir='/home/mprabhud/dataset/clevr_lang/npys/ab_5t.txt', hyp_N=2)
query_train_loader = DataLoader(query_train_dataset, batch_size=1, shuffle=True, collate_fn=collate_boxes)

Initialised..... 234  files...


In [ ]:
for j in range(3):
    for i, (feed_dict_q_, feed_dict_k_, metadata_) in enumerate((query_train_loader)):    
        feed_dict_n_lists = sample_same_scene_negs(feed_dict_q_, feed_dict_k_, metadata_, 2, 8)[0]
        break
    break

In [ ]:
indexes = [feed_n[1] for feed_n in feed_dict_n_lists]

In [ ]:
q_idx = metadata_["query_image_index"].item()
k_idx = metadata_["key_image_index"].item()

In [ ]:
q_idx

0

In [ ]:
k_idx


11

In [ ]:
indexes

[tensor([3]),
 tensor([0]),
 tensor([1]),
 tensor([12]),
 tensor([9]),
 tensor([15]),
 tensor([6]),
 tensor([17]),
 tensor([7]),
 tensor([10]),
 tensor([4]),
 tensor([5]),
 tensor([11]),
 tensor([2]),
 tensor([14]),
 tensor([16])]

In [ ]:
len(indexes)

16